# MARL 2D Soccer Training (Kaggle GPU)

Instructions:
- Add this repo as a Kaggle dataset (or upload the folder) so these files are present under `/kaggle/working`.
- Enable GPU in Kaggle settings.
- Run the cells top to bottom. Training logs to `train_metrics.csv`, weights to `weights/ppo_actor.weights.h5`.

Notes:
- Rendering is off; training runs headless.
- If TensorFlow isn’t available in the image, uncomment the TF install cell.
- Tweak `episodes`/`game_length` to fit your budget.

In [ ]:
# Optional: install TF if the Kaggle image is missing GPU TF. Usually TF 2.x is preinstalled on Kaggle GPU runtimes.
# !pip install -q tensorflow==2.12.0
!pip install -q tqdm pygame

In [ ]:
import os
from pathlib import Path
import matplotlib.pyplot as plt

from train_agents import train

# Ensure working directory contains the project files
print("cwd:", Path.cwd())
print("Files:", [p.name for p in Path.cwd().iterdir() if p.is_file()][:20])

# Training parameters
episodes = 800           # increase for stronger play
game_length = 500        # shorter for faster updates
num_per_team = 1         # set >1 for larger teams
lr = 3e-4
gamma = 0.99
smooth = 50
anchor_prob = 0.2        # fraction of games vs fixed bot
opponent_bot_diff = 0.7  # lower = easier bot
pool_size = 6
pool_latest_prob = 0.6
add_every = 50
add_threshold = -0.5
log_path = Path("train_metrics.csv")
weights_path = Path("weights/ppo_actor")  # will save as .weights.h5

# Run training
train(
    episodes=episodes,
    game_length=game_length,
    num_per_team=num_per_team,
    render=False,
    log_path=log_path,
    smooth=smooth,
    lr=lr,
    gamma=gamma,
    save_weights=weights_path,
    anchor_prob=anchor_prob,
    opponent_bot_diff=opponent_bot_diff,
    pool_size=pool_size,
    pool_latest_prob=pool_latest_prob,
    add_every=add_every,
    add_threshold=add_threshold,
)

In [ ]:
# Plot training metrics
import csv

episodes = []
rewards = []

with open("train_metrics.csv", newline="") as f:
    reader = csv.DictReader(f)
    for row in reader:
        episodes.append(int(row["episode"]))
        rewards.append(float(row["reward"]))

plt.figure(figsize=(8, 4.5))
plt.plot(episodes, rewards, label="Episode reward")
plt.xlabel("Episode")
plt.ylabel("Reward")
plt.title("Training progress")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()